# OpenML-CTR23 Regression Benchmark

This notebook benchmarks a variety of regression models on the `OpenML-CTR23 - A curated tabular regression benchmarking suite` benchmark suite.


# Setup Environment

In [ ]:
%pip install -Uq pip black blackcellmagic scikit-learn openml optuna pandas jupyter ipywidgets nbformat setuptools
%load_ext blackcellmagic

In [ ]:
# Control Models
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor

SEED = 42

models = {
    "Dummy": DummyRegressor(strategy="mean"),
    "RF": RandomForestRegressor(max_depth=2, random_state=SEED),
    "LR": LinearRegression(),
    "MLP": MLPRegressor(solver="sgd", random_state=SEED),
}

In [3]:
# Test Models
from sklearn.base import BaseEstimator, RegressorMixin, clone
from sklearn.ensemble import StackingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor


class ForestMLPRegressor(BaseEstimator, RegressorMixin):
    def __init__(
        self,
        n_trees=5,
        trees_max_depth=3,
        final_estimator=MLPRegressor(),
        final_estimator__hidden_layer_sizes=100,
        random_state=0,
    ):
        self.n_trees = n_trees
        self.trees_max_depth = trees_max_depth
        self.final_estimator = final_estimator
        self.final_estimator__hidden_layer_sizes = final_estimator__hidden_layer_sizes
        self.random_state = random_state

    def _make_stacking(self):
        return StackingRegressor(
            estimators=[
                (
                    f"tree_{i}",
                    DecisionTreeRegressor(
                        max_depth=self.trees_max_depth,
                        random_state=self.random_state + i,
                    ),
                )
                for i in range(self.n_trees)
            ],
            final_estimator=clone(self.final_estimator).set_params(
                hidden_layer_sizes=self.final_estimator__hidden_layer_sizes,
            ),
            passthrough=True,
        )

    def fit(self, X, y):
        self.model_ = self._make_stacking()
        self.model_.fit(X, y)
        return self

    def predict(self, X):
        return self.model_.predict(X)


test_models = {
    "TreeMLP": StackingRegressor(
        estimators=[("tree", DecisionTreeRegressor(random_state=SEED))],
        final_estimator=models["MLP"],
        passthrough=True,
    ),
    "ForestMLP": ForestMLPRegressor(final_estimator=models["MLP"], random_state=SEED),
}

models.update(test_models)

In [4]:
# Data Preprocessing
import numpy as np
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

numeric_transformer = Pipeline(
    [
        ("numeric_imputer", SimpleImputer(strategy="median")),
        ("numeric_scaler", StandardScaler()),
    ]
)
categorical_transformer = Pipeline(
    [
        ("categorical_imputer", SimpleImputer(strategy="most_frequent")),
        (
            "categorical_encoder",
            OneHotEncoder(handle_unknown="ignore", sparse_output=False),
        ),
    ]
)
preprocessor = make_column_transformer(
    (numeric_transformer, make_column_selector(dtype_include=np.number)),  # type: ignore
    (categorical_transformer, make_column_selector(dtype_include=["object", "category"])),  # type: ignore
    sparse_threshold=0.0,
)

models = {
    model_name: Pipeline([("preprocessor", preprocessor), ("model", model)])
    for model_name, model in models.items()
}

## Hyperparameter Optimization (HPO) Objective Functions

The following Python cell defines functions that create 'objectives' for Optuna. Each objective function encapsulates the logic for:
1. Defining the hyperparameter search space for a specific regression model.
2. Evaluating a set of hyperparameters using cross-validation on the training data.

These objective functions are then used by Optuna to find the best hyperparameter combination for each model.

In [5]:
# Hyperparameter Optimization Space
hpo_spaces = {
    "Dummy": {},
    "RF": {
        "n_estimators": ("int", {"low": 5, "high": 100}),
        "max_depth": ("int", {"low": 1, "high": 20}),
    },
    "LR": {},
    "MLP": {
        "hidden_layer_sizes": ("int", {"low": 5, "high": 100}),
    },
    "TreeMLP": {
        "tree__max_depth": ("int", {"low": 1, "high": 10}),
        "final_estimator__hidden_layer_sizes": ("int", {"low": 5, "high": 100}),
    },
    "ForestMLP": {
        "n_trees": ("int", {"low": 2, "high": 10}),
        "trees_max_depth": ("int", {"low": 1, "high": 10}),
        "final_estimator__hidden_layer_sizes": ("int", {"low": 5, "high": 100}),
    },
}

models = {
    model_name: (model, hpo_spaces[model_name]) for model_name, model in models.items()
}

In [6]:
# Hyperparameter Optimization Configuration
import optuna
from sklearn.base import BaseEstimator, RegressorMixin, clone
from sklearn.model_selection import cross_val_score

# Study configuration
hpo_config = {
    "n_trials": 3,
    "timeout": None,
    "show_progress_bar": True,
}

# Cross-validation parameters for HPO objective function
objective_cv_params = {"cv": 5, "scoring": "neg_mean_squared_error"}


def create_objective(model, hpo_space, X, y):
    def objective(trial: optuna.Trial) -> float:
        param_type_map = {
            "int": trial.suggest_int,
            "float": trial.suggest_float,
            "categorical": trial.suggest_categorical,
        }
        params = {
            f"model__{p}": param_type_map[typ](f"model__{p}", **kw)
            for p, (typ, kw) in hpo_space.items()
        }
        model.set_params(**params)
        return cross_val_score(model, X, y, **objective_cv_params).mean()

    return objective


class TemplateRegressor(RegressorMixin, BaseEstimator):
    def __init__(self, model, hpo_space):
        self.model = model
        self.hpo_space = hpo_space

    def fit(self, X, y):
        model = clone(self.model)
        optuna.logging.set_verbosity(optuna.logging.WARNING)
        study = optuna.create_study(sampler=optuna.samplers.TPESampler(seed=SEED))
        study.optimize(
            create_objective(model, self.hpo_space, X, y),
            **hpo_config,
        )
        model.set_params(**study.best_params)
        self.fitted_model_ = model.fit(X, y)
        return self

    def predict(self, X):
        return self.fitted_model_.predict(X)

In [7]:
# Define Benchmark Suite
import openml
from IPython.display import display

SUITE_ID = "8f0ea660163b436bbd4abd49665c7b1d"  # OpenML-CTR23
suite = openml.study.get_suite(SUITE_ID)
display(suite)
print(suite.description)

OpenML Benchmark Suite
ID..............: 353
Name............: OpenML-CTR23 - A curated tabular regression benchmarking suite
Status..........: active
Main Entity Type: task
Study URL.......: https://www.openml.org/s/353
# of Data.......: 35
# of Tasks......: 35
Creator.........: https://www.openml.org/u/30127
Upload Time.....: 2023-05-31 16:39:49

Inclusion Criteria:  

* There are between 500 and 100000 observations.
* There are less than 5000 features after one-hot encoding all categorical features.
* The dataset is not in a sparse format.
* The observations are i.i.d., which means that we exclude datasets that have time dependencies or require grouped data splits.
* The dataset comes with a source or reference that clearly describes it.
* We did not consider the dataset to be artificial, but allowed simulated datasets.
* The data is not a subset of a larger dataset.
* There is a numeric target variable with at least 5 different values.
* The dataset is not trivially solvable by a linear model, i.e. the training error of a linear model fitted to the whole data has an R2 of less than 1.
* The dataset does not have ethical concerns.
* The use of the dataset for benchmarking is not forbidden.

In addition to the datasets, the OpenML tasks also contain resampling splits, which were determined according to the following rule: If th

In [8]:
# Download tasks from the suite
N_TASKS = 1
tasks = openml.tasks.get_tasks(
    (suite.tasks or [])[:N_TASKS], download_data=True, download_qualities=True
)
display(tasks)

[OpenML Regression Task
 Task Type Description: https://www.openml.org/tt/TaskType.SUPERVISED_REGRESSION
 Task ID..............: 361234
 Task URL.............: https://www.openml.org/t/361234
 Estimation Procedure.: crossvalidation
 Target Feature.......: rings]

In [ ]:
import pandas as pd
from sklearn.model_selection import cross_validate

cv_params = {
    "scoring": ["neg_mean_squared_error", "r2"],
    "return_train_score": True,
}

runs = {}

for task in tasks:
    model_results = {}
    splits = task.download_split().split
    X, y = task.get_X_and_y(dataset_format="dataframe")  # type: ignore

    for model_name, model in models.items():
        print(f"Running {model_name=} on {task.id=}...")
        run = cross_validate(
            TemplateRegressor(*model),
            X,
            y,
            cv=[s[0] for s in splits[0].values()],  # Using pre-defined OpenML splits
            **cv_params,
        )
        model_results[model_name] = pd.DataFrame(run).mean(axis=0)
    runs[task.id] = model_results

In [10]:
# Aggregate Results
import pandas as pd

runs_df = pd.DataFrame(runs)
metrics = cv_params["scoring"]
metrics_df: dict[str, tuple[pd.DataFrame, pd.DataFrame]] = {}
for metric in metrics:
    metrics_df[metric] = (
        runs_df.map(lambda r: r[f"test_{metric}"]),
        runs_df.map(lambda r: r[f"train_{metric}"]),
    )

    # Use first metric for ranking
test_ranks_df = metrics_df[metrics[0]][0].rank(axis=0, ascending=False)
test_ranks_df["avg_rank"] = test_ranks_df.mean(axis=1)
test_ranks_df["std_rank"] = test_ranks_df.std(axis=1)

train_ranks_df = metrics_df[metrics[0]][1].rank(axis=0, ascending=False)
train_ranks_df["avg_rank"] = train_ranks_df.mean(axis=1)
train_ranks_df["std_rank"] = train_ranks_df.std(axis=1)

metrics_df["rank"] = (test_ranks_df, train_ranks_df)

# Display all metrics and ranks
for metric, (test, train) in metrics_df.items():
    display(f"test_{metric}", test, f"train_{metric}", train)

'test_neg_mean_squared_error'

""


'train_neg_mean_squared_error'

""


'test_r2'

...

""


'test_rank'

,avg_rank,std_rank


'train_rank'

,avg_rank,std_rank


In [11]:
# Single model evaluation
from sklearn.model_selection import cross_validate

task = tasks[0]
splits = task.download_split().split
X, y = task.get_X_and_y(dataset_format="dataframe")  # type: ignore
model = TemplateRegressor(*models["ForestMLP"])
run = cross_validate(
    model,
    X,
    y,
    cv=[s[0] for s in splits[0].values()][:3],  # Using pre-defined OpenML splits
    **cv_params,
)
display(pd.DataFrame(run).mean(axis=0))

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

fit_time                        19.015890
score_time                       0.009783
test_neg_mean_squared_error     -5.436803
train_neg_mean_squared_error    -3.664474
test_r2                          0.479871
train_r2                         0.647085
dtype: float64